In [4]:
%matplotlib inline
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import os
import matplotlib
import time
import sklearn.model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import time
# from sklearn.model_selection import train_test_splite
# from sklearn

Read Data

In [8]:
reviews=pd.read_csv('./reviews.csv')
all_list_ids=reviews['listing_id'].unique()
documents=[]
for id in all_list_ids:
    selector=reviews.listing_id==id
    docs_per_house=reviews.loc[selector,'comments']
    doc=''
    for i in docs_per_house:
        doc+=str(i)
    documents.append(doc)

NLP preprocessing

In [10]:
cv=CountVectorizer(max_df=0.8,strip_accents='unicode',
                   max_features=1500,
                   min_df=0.2,
                   stop_words='english')
X=cv.fit_transform(documents)

Save word_preprocessing model

In [6]:
from joblib import dump,load
dump(cv,'countVectorize_1500.joblib')
# cv=load('countVectorize.joblib')
X=cv.transform(documents)

Run LDA

In [11]:
lda = LatentDirichletAllocation(n_components=40, max_iter=200,
                                n_jobs=-1,
                                learning_method='batch',
#                                 learning_offset=50.,
                                random_state=0)

start=time.time()
lda.fit(X)
print('trainning time: ', str(time.time()-start)[:-5])
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
tf_feature_names = cv.get_feature_names()
print_top_words(lda, tf_feature_names, 10)
dump(lda,'lda_40topic.joblib')

trainning time:  2692.75382995
Topic #0: subway, metro, close, station, la, nyc, es, muy, downtown, safe
Topic #1: subway, away, restaurants, blocks, grocery, stores, safe, neighborhood, need, just
Topic #2: room, clean, nice, comfortable, bed, friendly, recommend, really, host, location
Topic #3: apartment, helpful, places, gave, really, subway, home, tips, local, area
Topic #4: brooklyn, subway, manhattan, neighborhood, park, easy, beautiful, lines, walking, walk
Topic #5: apartment, new, small, location, york, perfect, flat, subway, good, little
Topic #6: clean, parking, nice, definitely, space, comfortable, neighborhood, easy, needed, host
Topic #7: subway, station, train, right, close, away, manhattan, just, block, convenient
Topic #8: hosts, friendly, people, recommend, highly, welcoming, accommodating, super, kind, helpful
Topic #9: check, late, night, accommodating, early, flexible, host, arrived, time, let
Topic #10: location, apartment, perfect, easy, host, check, nyc, clean,

NameError: name 'dump' is not defined

In [32]:
# import lda
# model = lda.LDA(n_topics=20, n_iter=1500, random_state=1)
# model.fit(X)  # model.fit_transform(X) is also available
# topic_word = model.topic_word_  # model.components_ also works
# n_top_words = 8
# tf_feature_names = cv.get_feature_names()
# # print_top_words(lda, tf_feature_names, 10)
# for i, topic_dist in enumerate(topic_word):
#     topic_words = np.array(tf_feature_names)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
#     print('Topic {}: {}'.format(i, ', '.join(topic_words)))
# doc_topic = model.doc_topic_
# for i in range(10):
#      print("{} (top topic: {})".format( doc_topic[i]))

In [11]:
lda.components_.shape

(40, 1500)

In [12]:
topic_word_distribution=lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
word_topic_distribution=(topic_word_distribution/topic_word_distribution.sum(axis=0)).T
host_topic_distribution=lda.transform(X)
topic_host_distribution=(host_topic_distribution/host_topic_distribution.sum(axis=0)).T
word_host_distribution=word_topic_distribution.dot(topic_host_distribution)

In [39]:
k=tf_feature_names.index('clean')
i=word_host_distribution[k].argsort()
i

array([26242, 24497,  2549, ..., 27276, 33518, 35288])

In [1]:
all_list_ids

NameError: name 'all_list_ids' is not defined

In [40]:
for j in i[-20:]:
    print(j)
    print(documents[j])

35763
Great room, updated kitchen, very clean and perfect spot right next to a main subway stop to go all over New York. 
36762
The apartment has a great location right next to public transit. Gisselle is incredibly responsive and organized, and her place is beautiful, clean, and extremely spacious. 
29503
If you’re looking to escape the downtown bustle and have public transportation literally steps from you, this is your place! Michael’s roommates were very considerate of my space and the room itself has everything a single traveler needs- WiFi, phone charger, clean towels and smart TV. Major plus, Michael’s bed was to die for and many convenient stores in the area as well. Just a slight downside for my fellow women, there’s catcalling left and right once you step out of the apartment, but if you keep your earphones handy, you’ll be alright!Michael war sehr hilfsbereit und freundlich. Das Haus ist sehr nah an der Subway und direkt vor dem Haus ist eine Bushaltestelle, mit dem man eben

In [51]:
# tf_feature_names

In [31]:
word_host_distribution[,i]

array([1.29600293e-08, 1.73172118e-08, 2.40217440e-08, ...,
       5.59724899e-04, 7.04177139e-04, 7.61958036e-04])

In [59]:
documents[i[-4]]


'Tenho que deixar registrado minha imensa satisfação em ter ficado hospedado na casa do Ryan e da Manú. A casa é exatamente como consta nas fotos, contudo a surpresa vem no capricho, cuidado e limpeza que só são possíveis constatar ao vivo. Manú sempre muito prestativa, muito antes da data do check in sempre manteve contato enviando vídeos descrevendo como utilizar os equipamentos da cozinha, peças do banheiro, entre outras coisas. O bairro é super residencial e fica menos de 5 minutos andando da estação de metrô que leva a Manhattam. Muitos mercadinhos por perto, inclusive 24 horas. Só tenho a dizer que vivi momentos mágicos em Nova Iorque, e parte disso devo à Manu e Ryan que proporcionaram conforto e segurança para mim e para o Lucas na nossa viagem. Esperamos retornar em breve hehe!Quando vc viaja p NY uma das coisas mais complicadas é a hospedagem. Ou é caro demais ou não atende os seus requisitos como  privacidade, limpeza, proximidade do metrô, Wi-Fi grátis, hospitalidade do anf

In [47]:
selector=reviews['listing_id']==host_id
reviews.ix[selector,'comments'].values
# reviews

array(['For those in love with the vibrancy of the East Village, this is a fantastic apartment. The street itself is quiet at night, while surrounded by quaint shops and great restaurants and bars. \r\nMy interactions with Aviad were always first rate.'],
      dtype=object)